### 1. Import

In [ ]:
import os
import json

from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper
from datasets import load_dataset

from document_processor import TextProcessor, ImageProcessor, PageImageProcessor, ImageTextualSummaryProcessor, ImageTextualSummaryProcessorLarge
from multimodal_rag import MultimodalRAG
from embedder import OpenAIEmbedder, ColPaliEmbedder
from pdf_to_qa import generate_qa_for_pdf, generate_chartQA_pdf_and_json
from evaluation import evaluate_generation, evaluate_generation_chartQA, compute_mrr_at_k, compute_recall_at_k, compute_precision_at_k, compute_f1_score, compute_map_at_k

### 2. Configuration

In [ ]:
dataset = "CoinQA"
k = 10

if dataset.upper() == "COINQA":
    #PDF_FILE = "knowledge/subset_riksbanken.pdf"
    PDF_FILE = "knowledge/riksbanken.pdf"
elif dataset.upper() == "CHARTQA":
    #PDF_FILE = "knowledge/subset_ChartQA.pdf"
    PDF_FILE = "knowledge/ChartQA_human.pdf"
else:
    raise ValueError("Dataset not supported")

#text_processor = TextProcessor(OpenAIEmbedder())
#image_processor = ImageProcessor(ColPaliEmbedder(), dataset)
#page_image_processor = PageImageProcessor(ColPaliEmbedder())
image_textual_summary_processor = ImageTextualSummaryProcessor(OpenAIEmbedder(), dataset)

rag = MultimodalRAG([image_textual_summary_processor], PDF_FILE)
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini")) # For Ragas evaluation

##### Merge batches

In [ ]:
import os
import json
import torch

def merge_batches(batch_dir="embedding_cache/batches", output_dir="embedding_cache", name="ImageTextualSummaryProcessor"):
    all_embeddings = []
    all_metadata = []

    batch_files = sorted(os.listdir(batch_dir))
    embedding_files = [f for f in batch_files if f.endswith("_embeddings.pt")]

    for embed_file in embedding_files:
        base = embed_file.replace("_embeddings.pt", "")
        meta_file = f"{base}_metadata.json"

        emb = torch.load(os.path.join(batch_dir, embed_file))

        if isinstance(emb, list):
            for item in emb:
                if isinstance(item, torch.Tensor):
                    all_embeddings.append(item)
                elif isinstance(item, list):
                    if all(isinstance(x, float) for x in item):  # plain embedding
                        all_embeddings.append(torch.tensor(item))
                    elif all(isinstance(x, torch.Tensor) for x in item):  # list of tensors
                        all_embeddings.extend(item)
                    else:
                        raise TypeError(f"Unexpected item list contents: {item}")
                else:
                    raise TypeError(f"Unexpected embedding item: {type(item)}")
        elif isinstance(emb, torch.Tensor):
            all_embeddings.append(emb)
        else:
            raise TypeError(f"Unexpected embedding type: {type(emb)}")

        # Load metadata
        with open(os.path.join(batch_dir, meta_file)) as f:
            meta = json.load(f)
        all_metadata.extend(meta)

    merged_embeddings = torch.stack(all_embeddings, dim=0)

    torch.save(merged_embeddings, os.path.join(output_dir, f"{name}_merged_embeddings.pt"))
    with open(os.path.join(output_dir, f"{name}_merged_metadata.json"), "w") as f:
        json.dump(all_metadata, f)

    print(f"Merged {len(embedding_files)} batches into {output_dir} with {len(all_embeddings)} embeddings.")

if __name__ == "__main__":
    merge_batches()


### 3. Generate Dataset

In [ ]:
# Check if QA file already exists
qa_filepath = "json_files/QA_" + os.path.basename(PDF_FILE).replace('.pdf', '.json')

if os.path.exists(qa_filepath):
    print(f"Using existing QA file: {qa_filepath}")

elif dataset.upper() == "COINQA":
    generate_qa_for_pdf(PDF_FILE, json_output_path=qa_filepath, mode="per_image")
    print(f"Generated new CoinQA file: {qa_filepath}")

elif dataset.upper() == "CHARTQA":
    #chartqa = load_dataset('lmms-lab/ChartQA', split='test')
    #subset_chartqa = chartqa.select(range(20))
    
    generate_chartQA_pdf_and_json(subset_chartqa, pdf_output_path=PDF_FILE, json_output_path=qa_filepath)
    print(f"Generated new ChartQA file: {qa_filepath}")

### 4. Answering the QA

In [ ]:
with open(qa_filepath, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

# Generate dataset
rag_generated_answers = []

# Check if generated answers file already exists
rag_answers_path = "json_files/rag_generated_answers_" + os.path.basename(qa_filepath).replace('.json', f'_{rag.name}.json')

if os.path.exists(rag_answers_path):
    with open(rag_answers_path, 'r', encoding='utf-8') as f:
        rag_generated_answers = json.load(f)
    start_index = len(rag_generated_answers)
    print(f"Using existing RAG generated answers file: {rag_answers_path}")
    print(f"Resuming from question {start_index + 1}")
else:
    start_index = 0

batch_size = 20
total_questions = len(qa_data)
output_dataset_file = rag_answers_path

for i in range(start_index, total_questions):
    qa = qa_data[i]
    query = qa["question"]
    reference = qa["answer"]
    
    # Get relevant documents and generate answer
    relevant_docs = rag.get_most_relevant_docs(query, top_k=k)
    #response = rag.generate_answer_chartQA(query, relevant_docs)

    rag_generated_answers.append(
        {
            "query": query,
            "retrieved_contexts": relevant_docs,
            #"generated_answer": response,
            "true_answer": reference
        }
    )

    # Save intermediate results every batch_size questions
    if (i + 1) % batch_size == 0 or (i + 1) == total_questions:
        # Remove 'content' from retrieved_contexts before saving
        for item in rag_generated_answers:
            for idx, context in enumerate(item["retrieved_contexts"]):
                if idx > 0:
                    context.pop("content", None)

        with open(output_dataset_file, 'w', encoding='utf-8') as f:
            json.dump(rag_generated_answers, f, ensure_ascii=False, indent=4)
        print(f"Saved progress: {i + 1}/{total_questions} questions processed.")

print(f"Generated new RAG generated answers file: {output_dataset_file}")

##### 4.1 Evaluate retrieval right away

In [ ]:
rag_generated_answers = []

# Check if generated answers file already exists
rag_answers_path = "json_files/rag_generated_answers_" + os.path.basename(qa_filepath).replace('.json', f'_{rag.name}.json')

if os.path.exists(rag_answers_path):
    with open(rag_answers_path, 'r', encoding='utf-8') as f:
        rag_generated_answers = json.load(f)

with open(qa_filepath, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

### 5. Evaluate Retrieval

In [ ]:
from typing import List, Union, Tuple

def normalize_input(
    retrieved: Union[List[List[str]], List[str]], 
    true_pages: Union[List[List[str]], List[str]]  # was `real`
) -> Tuple[List[List[str]], List[List[str]]]:
    if isinstance(retrieved[0], str):  # Single query
        return [list(map(str, retrieved))], [list(map(str, true_pages))]
    else:
        return (
            [list(map(str, r)) for r in retrieved],
            [list(map(str, r)) for r in true_pages]
        )


def compute_mrr_at_k(
    all_retrieved_pages: Union[List[List[str]], List[str]], 
    all_real_pages: Union[List[List[str]], List[str]], 
    k: int
) -> float:
    all_retrieved_pages, all_real_pages = normalize_input(all_retrieved_pages, all_real_pages)
    reciprocal_ranks = []

    for retrieved_pages, real_pages in zip(all_retrieved_pages, all_real_pages):
        top_k_pages = retrieved_pages[:k]
        rank = float('inf')

        for real_page in real_pages:
            if real_page in top_k_pages:
                rank = min(rank, top_k_pages.index(real_page) + 1)  # 1-based

        reciprocal_ranks.append(1.0 / rank if rank != float('inf') else 0.0)

    return sum(reciprocal_ranks) / len(reciprocal_ranks) if reciprocal_ranks else 0.0


def compute_recall_at_k(
    all_retrieved_pages: Union[List[List[str]], List[str]], 
    all_real_pages: Union[List[List[str]], List[str]], 
    k: int
) -> float:
    all_retrieved_pages, all_real_pages = normalize_input(all_retrieved_pages, all_real_pages)
    total_hits = 0
    total_relevant = 0

    for retrieved_pages, real_pages in zip(all_retrieved_pages, all_real_pages): 
        top_k_pages = retrieved_pages[:k]
        hits = sum(1 for page in real_pages if page in top_k_pages)
        total_hits += hits
        total_relevant += len(real_pages)

    return total_hits / total_relevant if total_relevant else 0.0


def compute_precision_at_k(
    all_retrieved_pages: Union[List[List[str]], List[str]], 
    all_real_pages: Union[List[List[str]], List[str]], 
    k: int
) -> float:
    all_retrieved_pages, all_real_pages = normalize_input(all_retrieved_pages, all_real_pages)
    total_hits = 0
    total_retrieved = 0

    for retrieved_pages, real_pages in zip(all_retrieved_pages, all_real_pages):
        top_k_pages = retrieved_pages[:k]
        hits = sum(1 for page in top_k_pages if page in real_pages)
        total_hits += hits
        total_retrieved += len(top_k_pages)

    return total_hits / total_retrieved if total_retrieved else 0.0


def compute_f1_score(
    all_retrieved_pages: Union[List[List[str]], List[str]], 
    all_real_pages: Union[List[List[str]], List[str]], 
    k: int
) -> float:
    precision = compute_precision_at_k(all_retrieved_pages, all_real_pages, k)
    recall = compute_recall_at_k(all_retrieved_pages, all_real_pages, k)
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0


def compute_map_at_k(
    all_retrieved_pages: Union[List[List[str]], List[str]], 
    all_real_pages: Union[List[List[str]], List[str]], 
    k: int
) -> float:
    all_retrieved_pages, all_real_pages = normalize_input(all_retrieved_pages, all_real_pages)
    average_precisions = []

    for retrieved_pages, real_pages in zip(all_retrieved_pages, all_real_pages):
        top_k_pages = retrieved_pages[:k]
        num_relevant = 0
        precision_sum = 0.0

        for i, page in enumerate(top_k_pages):
            if page in real_pages:
                num_relevant += 1
                precision_sum += num_relevant / (i + 1)  # 1-based index

        average_precisions.append(precision_sum / num_relevant if num_relevant > 0 else 0.0)

    return sum(average_precisions) / len(average_precisions) if average_precisions else 0.0


all_real_pages, all_retrieved_pages = [], []

for rag_answer in rag_generated_answers:
    real_page = next(
        item["page_number"] for item in qa_data if item["question"] == rag_answer["query"]
    )
    
    #retrieved_pages = [doc["page_number"] for doc in rag_answer["retrieved_contexts"]]
    retrieved_pages = [str(doc["page_number"]) for doc in rag_answer["retrieved_contexts"]]

    all_real_pages.append([str(real_page)] if isinstance(real_page, (int, float)) else [str(p) for p in real_page])

    #all_real_pages.append([real_page] if isinstance(real_page, int) else real_page)
    all_retrieved_pages.append(retrieved_pages)

# Function to test a specific question by index
def test_question(index):
    if index < 1 or index > len(rag_generated_answers):
        print("Invalid index. Please select a number between 1 and 5.")
        return

    rag_answer = rag_generated_answers[index - 1]
    real_page = all_real_pages[index - 1]
    retrieved_pages = all_retrieved_pages[index - 1]

    print(f"Question: {rag_answer['query']}")
    print(f"True Answer: {rag_answer['true_answer']}")
    #print(f"Generated Answer: {rag_answer['generated_answer']}")
    print(f"Real Page(s): {real_page}")
    print(f"Retrieved Pages: {retrieved_pages}")
    return real_page, retrieved_pages

#Example usage: test the first question
#real_page, retrieved_pages = test_question(6)
# Or test everything
real_page, retrieved_pages = all_real_pages, all_retrieved_pages


mrr = compute_mrr_at_k(retrieved_pages, real_page, k)
print(f"MRR@{k}: {mrr:.4f}")
recall = compute_recall_at_k(retrieved_pages, real_page, k)
print(f"Recall@{k}: {recall:.4f}")
precision = compute_precision_at_k(retrieved_pages, real_page, k)
print(f"Precision@{k}: {precision:.4f}")
f1_score = compute_f1_score(retrieved_pages, real_page, k)
print(f"F1 Score@{k}: {f1_score:.4f}")
map_score = compute_map_at_k(retrieved_pages, real_page, k)
print(f"mAP Score: {map_score:.4f}")

In [ ]:
from typing import List, Union, Tuple

def normalize_input(
    retrieved: Union[List[List[str]], List[str]], 
    true_pages: Union[List[List[str]], List[str]]  # was `real`
) -> Tuple[List[List[str]], List[List[str]]]:
    if isinstance(retrieved[0], str):  # Single query
        return [list(map(str, retrieved))], [list(map(str, true_pages))]
    else:
        return (
            [list(map(str, r)) for r in retrieved],
            [list(map(str, r)) for r in true_pages]
        )


def compute_mrr_at_k(
    all_retrieved_pages: Union[List[List[str]], List[str]], 
    all_real_pages: Union[List[List[str]], List[str]], 
    k: int
) -> float:
    all_retrieved_pages, all_real_pages = normalize_input(all_retrieved_pages, all_real_pages)
    reciprocal_ranks = []

    for retrieved_pages, real_pages in zip(all_retrieved_pages, all_real_pages):
        top_k_pages = retrieved_pages[:k]
        rank = float('inf')

        for real_page in real_pages:
            if real_page in top_k_pages:
                rank = min(rank, top_k_pages.index(real_page) + 1)  # 1-based

        reciprocal_ranks.append(1.0 / rank if rank != float('inf') else 0.0)

    return sum(reciprocal_ranks) / len(reciprocal_ranks) if reciprocal_ranks else 0.0


def compute_recall_at_k(
    all_retrieved_pages: Union[List[List[str]], List[str]], 
    all_real_pages: Union[List[List[str]], List[str]], 
    k: int
) -> float:
    all_retrieved_pages, all_real_pages = normalize_input(all_retrieved_pages, all_real_pages)
    total_hits = 0
    total_relevant = 0

    for retrieved_pages, real_pages in zip(all_retrieved_pages, all_real_pages): 
        top_k_pages = retrieved_pages[:k]
        hits = sum(1 for page in real_pages if page in top_k_pages)
        total_hits += hits
        total_relevant += len(real_pages)

    return total_hits / total_relevant if total_relevant else 0.0


def compute_precision_at_k(
    all_retrieved_pages: Union[List[List[str]], List[str]], 
    all_real_pages: Union[List[List[str]], List[str]], 
    k: int
) -> float:
    all_retrieved_pages, all_real_pages = normalize_input(all_retrieved_pages, all_real_pages)
    total_hits = 0
    total_retrieved = 0

    for retrieved_pages, real_pages in zip(all_retrieved_pages, all_real_pages):
        top_k_pages = retrieved_pages[:k]
        hits = sum(1 for page in top_k_pages if page in real_pages)
        total_hits += hits
        total_retrieved += len(top_k_pages)

    return total_hits / total_retrieved if total_retrieved else 0.0


def compute_f1_score(
    all_retrieved_pages: Union[List[List[str]], List[str]], 
    all_real_pages: Union[List[List[str]], List[str]], 
    k: int
) -> float:
    precision = compute_precision_at_k(all_retrieved_pages, all_real_pages, k)
    recall = compute_recall_at_k(all_retrieved_pages, all_real_pages, k)
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0


def compute_map_at_k(
    all_retrieved_pages: Union[List[List[str]], List[str]], 
    all_real_pages: Union[List[List[str]], List[str]], 
    k: int
) -> float:
    all_retrieved_pages, all_real_pages = normalize_input(all_retrieved_pages, all_real_pages)
    average_precisions = []

    for retrieved_pages, real_pages in zip(all_retrieved_pages, all_real_pages):
        top_k_pages = retrieved_pages[:k]
        num_relevant = 0
        precision_sum = 0.0

        for i, page in enumerate(top_k_pages):
            if page in real_pages:
                num_relevant += 1
                precision_sum += num_relevant / (i + 1)  # 1-based index

        average_precisions.append(precision_sum / num_relevant if num_relevant > 0 else 0.0)

    return sum(average_precisions) / len(average_precisions) if average_precisions else 0.0


all_real_pages, all_retrieved_pages = [], []

for rag_answer in rag_generated_answers:
    real_page = next(
        item["page_number"] for item in qa_data if item["question"] == rag_answer["query"]
    )
    
    #retrieved_pages = [doc["page_number"] for doc in rag_answer["retrieved_contexts"]]
    retrieved_pages = [str(doc["page_number"]) for doc in rag_answer["retrieved_contexts"]]

    all_real_pages.append([str(real_page)] if isinstance(real_page, (int, float)) else [str(p) for p in real_page])

    #all_real_pages.append([real_page] if isinstance(real_page, int) else real_page)
    all_retrieved_pages.append(retrieved_pages)

# Function to test a specific question by index
def test_question(index):
    if index < 1 or index > len(rag_generated_answers):
        print("Invalid index. Please select a number between 1 and 5.")
        return

    rag_answer = rag_generated_answers[index - 1]
    real_page = all_real_pages[index - 1]
    retrieved_pages = all_retrieved_pages[index - 1]

    print(f"Question: {rag_answer['query']}")
    print(f"True Answer: {rag_answer['true_answer']}")
    #print(f"Generated Answer: {rag_answer['generated_answer']}")
    print(f"Real Page(s): {real_page}")
    print(f"Retrieved Pages: {retrieved_pages}")
    return real_page, retrieved_pages

#Example usage: test the first question
#real_page, retrieved_pages = test_question(6)
# Or test everything
real_page, retrieved_pages = all_real_pages, all_retrieved_pages


mrr = compute_mrr_at_k(retrieved_pages, real_page, k)
print(f"MRR@{k}: {mrr:.4f}")
recall = compute_recall_at_k(retrieved_pages, real_page, k)
print(f"Recall@{k}: {recall:.4f}")
precision = compute_precision_at_k(retrieved_pages, real_page, k)
print(f"Precision@{k}: {precision:.4f}")
f1_score = compute_f1_score(retrieved_pages, real_page, k)
print(f"F1 Score@{k}: {f1_score:.4f}")
map_score = compute_map_at_k(retrieved_pages, real_page, k)
print(f"mAP Score: {map_score:.4f}")

In [ ]:
import os
import json

# Config
PDF_FILE = "knowledge/riksbanken.pdf"
qa_filepath = "json_files/QA_" + os.path.basename(PDF_FILE).replace('.pdf', '.json')

# Paths to different RAG result files
rag_answer_paths = {
    "Txt sum (cmplx)": "json_files/rag_generated_answers_QA_riksbanken_dual_storage.json",
}

k_values = [1, 2, 3, 4, 5, 10]

# Load QA data once
with open(qa_filepath, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

# Prepare results storage
results = {}

# Loop through each RAG result file
for name, path in rag_answer_paths.items():
    with open(path, 'r', encoding='utf-8') as f:
        rag_generated_answers = json.load(f)

    all_real_pages, all_retrieved_pages = [], []

    for rag_answer in rag_generated_answers:
        real_page = next(
            item["page_number"] for item in qa_data if item["question"] == rag_answer["query"]
        )
        retrieved_pages = [str(doc["page_number"]) for doc in rag_answer["retrieved_contexts"]]

        all_real_pages.append([str(real_page)] if isinstance(real_page, (int, float)) else [str(p) for p in real_page])
        all_retrieved_pages.append(retrieved_pages)

    results[name] = []
    for k in k_values:
        f1_at_k = compute_f1_score(all_retrieved_pages, all_real_pages, k)
        results[name].append(f1_at_k)

# Print results in a formatted table
print(f"{'Method':<25} | " + " | ".join([f"F1@{k:<3}" for k in k_values]))
print("-" * (27 + 9 * len(k_values)))

for name, scores in results.items():
    score_strs = [f"{score:.4f}" for score in scores]
    print(f"{name:<25} | " + " | ".join(score_strs))


### 6.1 Batch evaluation

In [ ]:
import os
import json

# Define the file path for storing the results
results_folder = "resultsbatch"
os.makedirs(results_folder, exist_ok=True)

# Function to evaluate in batches
def evaluate_in_batches(rag_generated_answers, evaluator_llm, batch_size=25):
    total_metrics = {"faithful_rate": 0.0, "relevance_rate": 0.0, "answer_correctness": 0.0}
    total_batches = len(rag_generated_answers) // batch_size + (1 if len(rag_generated_answers) % batch_size > 0 else 0)
    
    for batch_index in range(total_batches):
        # Define the batch's results file path
        batch_file_path = os.path.join(results_folder, f"batch_{batch_index+1}_{rag.name}.json")
        
        # Check if the batch result already exists
        if os.path.exists(batch_file_path):
            print(f"Batch {batch_index+1} results already exist: {batch_file_path}")
            with open(batch_file_path, 'r', encoding='utf-8') as file:
                batch_results = json.load(file)
                print(f"Loaded batch {batch_index+1} results: {batch_results}")
        else:
            # Create a subset of the answers for this batch
            batch_start = batch_index * batch_size
            batch_end = min((batch_index + 1) * batch_size, len(rag_generated_answers))
            subset_rag_generated_answers = rag_generated_answers[batch_start:batch_end]
            
            # Evaluate the batch
            faithfulness_and_relevance = str(evaluate_generation(subset_rag_generated_answers, evaluator_llm))
            
            # Replace single quotes with double quotes
            json_string = faithfulness_and_relevance.replace("'", '"')
            
            # Save the batch results
            with open(batch_file_path, 'w', encoding='utf-8') as f:
                f.write(json_string)  # Write string instead of using json.dump()
            print(f"Results for batch {batch_index+1} saved to: {batch_file_path}")
            
            # Load the batch results into the dictionary
            batch_results = json.loads(json_string)
        
        # Accumulate metrics for averaging
        total_metrics["faithful_rate"] += batch_results.get("faithful_rate", 0)
        total_metrics["relevance_rate"] += batch_results.get("relevance_rate", 0)
        total_metrics["answer_correctness"] += batch_results.get("answer_correctness", 0)
    
    # Calculate the average of the metrics
    total_batches = max(1, total_batches)  # Avoid division by zero in case of empty input
    average_metrics = {
        "average_faithful_rate": total_metrics["faithful_rate"] / total_batches,
        "average_relevance_rate": total_metrics["relevance_rate"] / total_batches,
        "average_answer_correctness": total_metrics["answer_correctness"] / total_batches
    }

    return average_metrics

# Example usage of the function
if __name__ == "__main__":
    # Assuming rag_generated_answers and evaluator_llm are already defined
    subset_rag_generated_answers = rag_generated_answers[:1]  # Example subset

    print(subset_rag_generated_answers)
    average_metrics = evaluate_in_batches(subset_rag_generated_answers, evaluator_llm)
    print("Final average metrics:", average_metrics)


### 6.2 Evaluate Generation

In [ ]:
# Define the file path for storing the results
results_folder = "results"
os.makedirs(results_folder, exist_ok=True)
results_file_path = os.path.join(results_folder, os.path.basename(qa_filepath).replace('.json', f'_{rag.name}.json'))

# Create a subset of 20 answers from rag_generated_answers
subset_rag_generated_answers = rag_generated_answers[:1]
# Check if the results file already exists
if os.path.exists(results_file_path):
    print(f"Results already exist: {results_file_path}")
    print("Loading existing results...")
    with open(results_file_path, 'r', encoding='utf-8') as file:
        print(file.read())
    try:
        with open(results_file_path, 'r', encoding='utf-8') as f:
            faithfulness_and_relevance = json.load(f)
    except json.JSONDecodeError:
        print(f"Error: The file {results_file_path} contains invalid JSON or is empty.")
        faithfulness_and_relevance = None
else:
    # Evaluate generation
    #faithfulness_and_relevance = str(evaluate_generation_chart(rag_generated_answers, evaluator_llm))
    faithfulness_and_relevance = str(evaluate_generation(subset_rag_generated_answers, evaluator_llm))
    
    # Replace single quotes with double quotes
    json_string = faithfulness_and_relevance.replace("'", '"')

    with open(results_file_path, 'w', encoding='utf-8') as f:
        f.write(json_string)  # Write string instead of using json.dump()
    print(f"Results saved to: {results_file_path}")